In [1]:
%pylab qt5
%matplotlib inline
import scipy.stats

Populating the interactive namespace from numpy and matplotlib


/home/ca/.local/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['degrees', 'cos', 'exp', 'pi', 'hypot', 'tan', 'log10', 'ldexp', 'log', 'copysign', 'modf', 'fabs', 'isfinite', 'isinf', 'nan', 'ceil', 'inf', 'cosh', 'fmod', 'e', 'log2', 'gamma', 'trunc', 'isclose', 'radians', 'sinh', 'frexp', 'tanh', 'floor', 'sin', 'isnan', 'sqrt', 'log1p', 'expm1']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [ ]:
pylab.rcParams['figure.figsize'] = (10, 6)

In [ ]:
wasser1 = loadtxt('./wasser1.csv', delimiter=',', skiprows=1)
wt1, wR1 = wasser1[:, 0], wasser1[:, 1]
wasser2 = loadtxt('./wasser2.csv', delimiter=',', skiprows=1)
wt2, wR2 = wasser2[:, 0], wasser2[:, 1]
wasser3 = loadtxt('./wasser3.csv', delimiter=',', skiprows=1)
wt3, wR3 = wasser3[:, 0], wasser3[:, 1]

In [ ]:
plt.plot(wt1, wR1, '+', wt2, wR2, '.',  wt3, wR3, '1')
plt.plot(linspace(0, 255, 50), [6.6633]*50,  'k--')

In [ ]:
salz1 = loadtxt('./salz1.csv', delimiter=',', skiprows=1)
st1, sR1 = salz1[:, 0], salz1[:, 1]
salz2 = loadtxt('./salz2.csv', delimiter=',', skiprows=1)
st2, sR2 = salz2[:, 0], salz2[:, 1]
salz3 = loadtxt('./salz3.csv', delimiter=',', skiprows=1)
st3, sR3 = salz3[:, 0], salz3[:, 1]

In [ ]:
plt.plot(st1, sR1, '+', st2, sR2, '.',  st3, sR3, '1')
plt.plot(linspace(0, 755, 50), [7.0233]*50,  'k--')

In [ ]:
salz3[-1]

## Kalibrierung

In [ ]:
kali = loadtxt('./kalibrierung.csv', delimiter=',', skiprows=1)

In [ ]:
Temp = kali[:, 0]
R = kali[:, 1]

In [ ]:
a, b, res, _, err_a = scipy.stats.linregress(R, Temp)
a, b, err_a

In [ ]:
err_b = 0.174 # aus Origin

In [ ]:
plt.plot(R, Temp, 'o')
x = linspace(min(R), max(R))
plt.plot(x, a*x + b)

$T = a R + b$

In [ ]:
T = vectorize(lambda R: a*R + b)

Wir können auch eine Korrektur zweiter Ordnung an die Regression berechnen:

In [ ]:
# AA = transpose([ones_like(R), R, R**2])
# m2, [res2], _, _ = linalg.lstsq(AA, Temp)
# b2, a2, k = m2
# a2, b2, k, res2

In [ ]:
# plt.plot(R, Temp, 'o')
# plt.plot(x, a2*x + b2 + k* x**2)

Am besten fitten wir jedoch einen exponentiellen Zusammenhang, denn das ist was tatsächlich vorliegt...

$R = \beta \, e^{\alpha T} \quad\Longrightarrow\quad \log R = \log\beta + \alpha T$

Da die Werte von $R$ nicht zu stark streuen (bzw. nicht zu nah an $0$ kommen wo die Ableitung von $\log$ singulär wird), kommen wir wohl ohne Gewichtung klar...

In [ ]:
α, logβ , _, _, err_α = scipy.stats.linregress(Temp, log(R))
β = exp(logβ)
print(f'α = {α} +- {err_α}, β = {β}')
y = linspace(min(Temp), max(Temp))
#plt.yscale('log')
_ = plt.plot(R, Temp, 'o')
_ = plt.plot(β * exp(α * y), y)
_ = plt.plot(x, a*x + b, ':')

$T = \dfrac1{\alpha} \left(\log R - \log\beta \right)$

In [ ]:
Texp = vectorize(lambda R: 1/α * (log(R) - log(β)))

### Temperatur-Zeit Diagramme

In [ ]:
_ = plt.plot(wt1, T(wR1), '+', wt2, T(wR2), '.',  wt3, T(wR3), '1')
_ = plt.plot(linspace(0, 255, 50), [T(6.6633)]*50,  'k--')
_ = plt.title('Temperatur - Zeit')
_ = plt.ylabel('Temperatur in °C')
_ = plt.xlabel('Zeit in s')
T0 = T(6.6633)
T0exp = Texp(6.6633)
print(f'Temperatur: {T0} °C')
print(f'mit exp. fit: {T0exp} °C')
print()

In [ ]:
_ = plt.plot(st1, T(sR1), '+', st2, T(sR2), '.',  st3, T(sR3), '1')
_ = plt.plot(linspace(0, 755, 50), [T(7.0233)]*50,  'k--')
_ = plt.title('Temperatur - Zeit')
_ = plt.ylabel('Temperatur in °C')
_ = plt.xlabel('Zeit in s')
T1 = T(7.0233)
T1exp = Texp(7.0233)
print(f'Temperatur: {T1} °C')
print(f'mit exp. fit: {T1exp} °C')
print()

In [ ]:
# in Gramm
# Masse des Wassers
m_W = 46.9183 - 26.7474 # Masse des Reagenzglases abziehen

m_S = 0.5700 # Masse des Salzes
# molare Masse des Salzes
M_S = 22.98977 + 14.0067 + 3 * 15.9994# NaNO3 Natrium: 22.98977, Stickstoff: 14.0067, Sauerstoff: 15.9994
N = m_S / M_S # gelöste Stoffmenge
N

K_G1 = 1.859 # K mol^-1

In [ ]:
M_S

In [ ]:
DT_G = T0 - T1
DT_G

In [ ]:
DT_Gexp = T0exp - T1exp
DT_Gexp

In [ ]:
# in SI
N_dis = DT_G * m_W * 1e-3 / K_G1
N_dis

In [ ]:
alpha = (N_dis / N - 1) / (2 - 1) # z = 2
alpha

In [ ]:
alpha_exp = DT_Gexp * m_W * 1e-3 / K_G1 / N - 1
alpha_exp

# Fehlerrechnung

Standardabweichung des mittelwerts für $R_0$ (destilliertes Wasser)  
Korrekturfaktor `t = 1.2` für wenige Messwerte

In [ ]:
wR1[-1], wR2[-1], wR3[-1]

In [ ]:
R0 = mean([6.67, 6.66, 6.66])
err_R0 = 1.2 * std([6.67, 6.66, 6.66], ddof=1) / sqrt(3)
R0, err_R0

Standardabweichung des mittelwerts für $R_1$ (Salzwasser)  
Korrekturfaktor `t = 1.2` für wenige Messwerte

In [ ]:
sR1[-1], sR2[-1], sR3[-1]

In [ ]:
R1 = mean([7.03, 7.02, 7.02])
err_R1 = 1.2 * std([7.03, 7.02, 7.02], ddof=1) / sqrt(3)
R1, err_R1

Die Numerik spackt mal wieder rum:

In [ ]:
(err_R0 - err_R1) / err_R0

In [ ]:
a, err_a

$\Delta T_G = T_0 - T_1 = a \cdot (R_0 - R_1)$

Fehler:
$\Delta(\Delta T_G) = \sqrt{(R_0 - R_1)^2 \Delta a^2 + a^2 \cdot ({\Delta R_0}^2 + {\Delta R_1}^2)}$

In [ ]:
err_DT_G = sqrt((R0 - R1)**2 * err_a**2 + a**2 * (err_R0**2 + err_R1**2))
DT_G, err_DT_G
print(f'Fehler aus a {abs((R0 - R1) * err_a):e}')
print(f'Fehler aus R0: {abs(a * err_R0):e}')
print(f'Fehler aus R1: {abs(a * err_R1):e}')

In [ ]:
# Fehler für die Schmelzpunkte
err_T0 = sqrt(a**2 * err_R0**2 + (err_a * R0 + err_b)**2)
err_T0
err_T1 = sqrt(a**2 * err_R1**2 + (err_a * R1 + err_b)**2)
err_T1

#### Exponentiell

$\Delta T_G = T_0 - T_1 = 1/\alpha \cdot (\log R_0 - \log R_1)$

Fehler:
$$\Delta(\Delta T_G) = \sqrt{(\log R_0 - \log R_1)^2 \frac{\Delta \alpha^2}{\alpha^4} +
\frac1{\alpha^2} \cdot \left( \frac{{\Delta R_0}^2}{{R_0}^2} + \frac{{\Delta R_1}^2}{{R_1}^2} \right)}$$

In [ ]:
err_DT_Gexp = sqrt((log(R0) - log(R1))**2 * err_α**2 / α**4 + 
                α**(-2) * (err_R0**2 / R0**2 + err_R1**2 / R1**2))
DT_Gexp, err_DT_Gexp
print(f'Fehler aus α {(log(R0) - log(R1)) * err_α / α**2:e}')
print(f'Fehler aus R0: {1/α * err_R0 / R0:e}')
print(f'Fehler aus R1: {1/α * err_R0 / R1:e}')

In [ ]:
err_waage = 0.0005 # in gramm
ERR_waage = 0.0002

In [ ]:
err_m_W = sqrt(2 * err_waage**2); ERR_m_W = 2 * ERR_waage
m_W, err_m_W, ERR_m_W
err_m_S = err_waage; ERR_m_S = ERR_waage

In [ ]:
err_N = err_m_S / M_S; ERR_N = ERR_m_S / M_S

In [ ]:
# N_dis = DT_G * m_W * 1e-3 / K_G1
N_dis, sqrt((m_W * 1e-3 / K_G1)**2 * err_DT_G**2 + (DT_G * 1e-3 / K_G1)**2 * ERR_m_W**2 )
print(f'Fehler aus DT_G {(m_W * 1e-3 / K_G1) * err_DT_G:e}')
print(f'Fehler aus m_W: {(DT_G * 1e-3 / K_G1) * ERR_m_W:e}')

Wir können den statistischen Fehler aus `m_W` getrost vernachlässigen, den systematischen auch, da er in der gleichen Größenordnung liegt

In [ ]:
err_N_dis = (m_W * 1e-3 / K_G1) * err_DT_G
N_dis, err_N_dis

In [ ]:
# alpha = (N_dis / N - 1) / (2 - 1) = N_dis / N - 1
err_alpha = sqrt(1/N**2 * err_N_dis**2 / 2**2 + N_dis**2 / N**4 * err_N**2 / 2**2)
print(f'Fehler aus N_dis {1/N * err_N_dis:e}')
print(f'Fehler aus N: {N_dis / N**2 * err_N:e}')
alpha, err_alpha

Wir können statisfischen den Fehler aus `N` vernachlässigen, den Systematischen auch, da er die gleiche Größenordnung hat

In [ ]:
alpha, 1/N *err_N_dis / 2